

**EXPLORATION AND CLEANING Neo_bank**

EXPLORATION Neo_bank Table DEVICES

In [15]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
from pandas_gbq import read_gbq

project_id = "data-analytics-bootcamp-363212"


query = """
SELECT *
FROM `data-analytics-bootcamp-363212.neo_bank.devices`
"""

df_devices = pd.read_gbq(query, project_id="data-analytics-bootcamp-363212")
df_devices.head()



/tmp/ipython-input-3088058791.py:15: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_devices = pd.read_gbq(query, project_id="data-analytics-bootcamp-363212")


,string_field_0,string_field_1
0,Apple,user_6809
1,Apple,user_3636
2,Apple,user_510
3,Apple,user_3468
4,Apple,user_7036


In [16]:
df_devices.shape

(19431, 2)

In [17]:
df_devices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19431 entries, 0 to 19430
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   string_field_0  19431 non-null  object
 1   string_field_1  19431 non-null  object
dtypes: object(2)
memory usage: 303.7+ KB


In [18]:
df_devices.isna().sum()

,0
string_field_0,0
string_field_1,0


In [19]:
df_devices.value_counts("string_field_0")

,count
string_field_0,
Android,9714
Apple,9673
Unknown,43
brand,1


In [20]:
# unicité primary key
df_devices["string_field_1"].nunique(), df_devices["string_field_1"].duplicated().sum()


(19431, np.int64(0))

CLEANING Neo_bank Table DEVICES

In [21]:
#Rename the columns

In [22]:
df_devices = df_devices.rename(columns={"string_field_1": "user_id","string_field_0": "device_brand"})

In [23]:
# Replace "brand" by "Unknow"

In [24]:
df_devices["device_brand"] = df_devices["device_brand"].replace({"brand": "Unknown"})

In [25]:
df_devices["device_brand"].value_counts(dropna=False)

,count
device_brand,
Android,9714
Apple,9673
Unknown,44


In [26]:
#Replace "Unknow" by "Not specified"

In [28]:
df_devices["device_brand"] = df_devices["device_brand"].replace("Unknown", "Not Specified")

In [29]:
df_devices["device_brand"].value_counts(dropna=False)

,count
device_brand,
Android,9714
Apple,9673
Not Specified,44


In [30]:
df_devices["user_id"].nunique(), df_devices["user_id"].duplicated().sum()

(19431, np.int64(0))

No duplicated user_id in devices.
However, devices table contains one extra user_id compared to users table (19,431 vs 19,430).

In [31]:
query = """
SELECT *
FROM `data-analytics-bootcamp-363212.neo_bank.users`
"""

df_users = pd.read_gbq(query, project_id="data-analytics-bootcamp-363212")
df_users.head()

/tmp/ipython-input-1475015310.py:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_users = pd.read_gbq(query, project_id="data-analytics-bootcamp-363212")


,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals
0,user_2811,1988,IE,C,2018-03-04 21:46:39.616834+00:00,0,STANDARD,NaN,NaN,0,0,0
1,user_4750,1987,FR,Ry,2018-04-07 06:49:18.345736+00:00,0,STANDARD,NaN,NaN,0,0,0
2,user_17686,1984,GB,Ayr,2018-12-04 10:28:02.653147+00:00,0,PREMIUM,NaN,NaN,6,0,0
3,user_18779,1965,GB,Ayr,2018-12-31 07:42:19.353613+00:00,0,STANDARD,NaN,NaN,0,0,0
4,user_7823,1999,PL,Buk,2018-06-07 03:22:40.590209+00:00,0,STANDARD,NaN,NaN,0,0,0


In [32]:
#Retrieve the unique 'user_id' values ​​as a list in order to find the value that is not in the USERS table.
devices_ids = set(df_devices["user_id"].dropna().unique())
users_ids = set(df_users["user_id"].dropna().unique())

In [33]:
missing_ids = devices_ids - users_ids
df_devices[df_devices["user_id"].isin(missing_ids)]


,device_brand,user_id
9673,Not Specified,user_id


One invalid row was found in the devices table where user_id equals the string 'user_id' (likely a header ingestion issue). I will remove this row.

In [34]:
df_devices = df_devices[df_devices["user_id"] != "user_id"]

After removing the invalid row, I re-ran basic data quality checks to confirm that user_id contains no missing values and remains unique (no duplicates)

In [35]:
df_devices["user_id"].isna().sum()


np.int64(0)

In [36]:
df_devices["user_id"].nunique(), df_devices["user_id"].duplicated().sum()


(19430, np.int64(0))

In [37]:
#I will check that both tables contain the exact same set of user_id values and that the user_id format is consistent in both datasets.

In [38]:
devices_ids = set(df_devices["user_id"].dropna().unique())
users_ids = set(df_users["user_id"].dropna().unique())

devices_ids == users_ids


True

In [39]:
df_devices["user_id"].dtype

dtype('O')

In [40]:
df_users["user_id"].dtype

dtype('O')